In [ ]:
import requests
from bs4 import BeautifulSoup

requests.packages.urllib3.util.connection.HAS_IPV6 = False

In [2]:
import sys
import locale

print(sys.getfilesystemencoding())
print(locale.getpreferredencoding())

utf-8
UTF-8


In [2]:
def get_headers_and_cookies() -> tuple[dict, dict]:
    headers = {
        "Accept-Language": "en-US,de;q=0.5",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": "JSESSIONID=EC04190F82889D8C9BAC752AF8838646; _abck=0A9A0316B6A3A24D1238C1B14D26C517~-1~YAAQJXvdWGMRP6SNAQAAwEXDsAsN3iftN8ALI7qy1QjQPqObR2DQkeyB8CV6tmfySmcD7MLQ8p1q9F2c3tiQoGufNswA3Ju5L7+UeHx6Wmnn0AwEoA7BGn+rr3GG34duNTwQW8EoF4HsIIeZ18UAFz/BhyecMVKiFta+y0QnyjNNNGE2RTjNbr5NcGPk9myqCFtFRe5Iih+rIVvFiwFwemB/+sYcaXkxST88+KZYjKtdI23ikS1L8/sLX9rmAH4u13fe6FJJtp3z0YRDp74WPYjXIiY4zmGbSbZHxiK6P48tTW8KOB7WwUGpbcO/iP7CVbFdfcIgsj60bS2pT1IBQ0XEb9VpZKjc0t61I9q3A/nODtJ23rOfp7kfW4gh+oMNh+XLlmzuiCKhHGQYUYGSuWLt~-1~-1~1708071203…qBOZJZXxS/r9NLJqRbZiqXnDJOmz7W/z1WYZW9YBPtOUdDUfvbmQ5JRvDNk7gMGsUExEYSGq5e1QXk5wxU+maWy7E9UpFjOJ6Mm0Q=~1; bm_mi=5A6D3F02F44132787908D05660DF17C7~YAAQJXvdWGsGP6SNAQAAOxPCsBat+dfY4XhYpEkjiISYNtfrgbflx6plwX5cwY9SGPPufuQh2VEeYBSXtFkiYBLq7SUnL3z0gnRMQzMxjIQAK1GvYLsHpS0/2EOvegIaDgg5CWzEvhoIGx/cJcIXCpzRrkmfKhE3sCT/6RH72Ri0CDtRFOZ1/BxSIsJhXQexw04R3WqLNX/pSI4XvWBpCjLrPrXPLHBnJLfT4vajkGKcibVn8081rtiIa/g3xX3oB4pk7G8TxdgwV19QQJftUc0nWSA7CyODsz/iS0QvH6unX2UxQSObZnE4jHzS4+fvQ/elnzohb+7ZKTkSGQW6ZlhuA7dCW2z9WyFEkmqYX6mF~1",
        "Host": "planetarium-hamburg.eventim-inhouse.de",
        "Pragma": "no-cache",
        "Referer": "https://planetarium-hamburg.eventim-inhouse.de/webshop/",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:122.0) Gecko/20100101 Firefox/122.0",
        # tell server to use utf-8
        "Accept-Charset": "utf-8"
        }

    cookies = {
        "_abck": "0A9A0316B6A3A24D1238C1B14D26C517~-1~YAAQJXvdWGMRP6SNAQAAwEXDsAsN3iftN8ALI7qy1QjQPqObR2DQkeyB8CV6tmfySmcD7MLQ8p1q9F2c3tiQoGufNswA3Ju5L7+UeHx6Wmnn0AwEoA7BGn+rr3GG34duNTwQW8EoF4HsIIeZ18UAFz/BhyecMVKiFta+y0QnyjNNNGE2RTjNbr5NcGPk9myqCFtFRe5Iih+rIVvFiwFwemB/+sYcaXkxST88+KZYjKtdI23ikS1L8/sLX9rmAH4u13fe6FJJtp3z0YRDp74WPYjXIiY4zmGbSbZHxiK6P48tTW8KOB7WwUGpbcO/iP7CVbFdfcIgsj60bS2pT1IBQ0XEb9VpZKjc0t61I9q3A/nODtJ23rOfp7kfW4gh+oMNh+XLlmzuiCKhHGQYUYGSuWLt~-1~-1~1708071203",
        "ak_bmsc": "D924247C76E45C4105AD2750BE06B7D5~000000000000000000000000000000~YAAQJXvdWBkGP6SNAQAANgvCsBa17Ah5rIiQTv8BXQYptq9xT32TJ5gFLpMaBoQemUSg0Y9j8Oym2wPBIawnNgiDUcoBPzswQD/2Rw1GKH9ghN0Ute7TUOXeZnE9Q5wkzGML5Wh6vl6jsD9SUO9ZmiSA+v11kWUDomuRYRiQIK95oESGXuU4AtSXygqsEviKgbUepF/dSELQiDWZEBlaVlMhiKaWRP4gBMPGLdaO/kETV6AvmOPGdXtqmChe5r92kuPGrcL0Adz/7cr2LwCWomzxK/QVF6aUo7eSOyx0fUh4aYWwbU1ceiIFMrF9rhXsFBNTkrhA2XxfAMxF520n6xLXKGQ7OQfGeYg9T4ttyVLrFmkVohwrFBdPpHouxGR/jSWWM1iQJhdFiwo3RH1zB6c=",
        "BIGipServer~ASP~sg_inhouse_planetarium-hamburg": "rd5o00000000000000000000ffff0ac05165o10063",
        "bm_mi": "5A6D3F02F44132787908D05660DF17C7~YAAQJXvdWGsGP6SNAQAAOxPCsBat+dfY4XhYpEkjiISYNtfrgbflx6plwX5cwY9SGPPufuQh2VEeYBSXtFkiYBLq7SUnL3z0gnRMQzMxjIQAK1GvYLsHpS0/2EOvegIaDgg5CWzEvhoIGx/cJcIXCpzRrkmfKhE3sCT/6RH72Ri0CDtRFOZ1/BxSIsJhXQexw04R3WqLNX/pSI4XvWBpCjLrPrXPLHBnJLfT4vajkGKcibVn8081rtiIa/g3xX3oB4pk7G8TxdgwV19QQJftUc0nWSA7CyODsz/iS0QvH6unX2UxQSObZnE4jHzS4+fvQ/elnzohb+7ZKTkSGQW6ZlhuA7dCW2z9WyFEkmqYX6mF~1",
        "bm_sv": "46A9F136F115FC7F326AFC6B0CC167BA~YAAQJXvdWGwGP6SNAQAAOxPCsBYnfePrc/t4W7W+lhGubSn3DU9GO0BOM9u9z4l2nCnt/0PUzBkWtaJclj2g0WAAXyrgSZR3L6YWg7rBh4vI0wEIE/Iy9cagyfrAKtshPlMMCintOBiJbS2IndubBq3+imnvsKnMM56oR7qBOZJZXxS/r9NLJqRbZiqXnDJOmz7W/z1WYZW9YBPtOUdDUfvbmQ5JRvDNk7gMGsUExEYSGq5e1QXk5wxU+maWy7E9UpFjOJ6Mm0Q=~1",
        "bm_sz": "EEAC53921DF6096419FA3722D33F614C~YAAQJXvdWG0GP6SNAQAAOxPCsBaVQ2Ry7JJmxAsfrSpVafjRaR9o/wSTbZUEMXcYKf0DQvX/p1Npx+fU8WlE+uoWTYzVWMS1KvNTZRmPhjj8Bk9FZnrYfYjwnTaufm+IxqKWCbKNizqYbBFsPIEXqQ/24lzCdXdSex0toD1wOcolwhUk/F7I8si+pFbbUju1yUEsi4mp7tGW0/gbYtBwMjf23pmnQmDk3+/grXVcErdk4qzdWEPEOwPpm/H+lPAEnPxgXFyHWydEDuGp/60KAGscEDnYLSneNGwygc3MznbPUYt9Hde57iielzZLbe41Zoagx1xSLZLRY722QiW47iIcUTmtEYinGQ7ev+wxNTeB1MWhhkk6bIlPxRvqJmzDXxPvq+9KThhYRUs=~4474166~3553328",
        "JSESSIONID": "EC04190F82889D8C9BAC752AF8838646"
    }

    headers_new = {}
    for key, value in headers.items():
        key = key.encode("utf-8")
        value = value.encode("utf-8")
        headers_new[key] = value

    cookies_new = {}
    for key, value in cookies.items():
        key = key.encode("utf-8").decode("utf-8")
        value = value.encode("utf-8").decode("utf-8")
        cookies_new[key] = value
    
    return headers_new, cookies_new

In [3]:
def get_event_list():
    url = "https://planetarium-hamburg.eventim-inhouse.de/webshop/webticket/eventlist"

    headers, cookies = get_headers_and_cookies()

    response = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.content, "html.parser")

    event_list = soup.find("ul", {"id": "eventListUl"}).find_all("li")

    return event_list

def get_event_info(event):
    event_info = {}
    event_info["title"] = event.find("h2").text
    event_info["id"] = event_id = event.find("h2")["id"].split("-")[-1]

    date_span = event.find("span", {"id": "event-date-" + event_id})

    event_info["weekday"] = date_span.find("span").text.strip()
    event_info["date"] = date_span.text.split("</span>")[0].split(" ")[-1].strip()

    event_info["time_HH_MM"] = event.find("span", {"id": "event-time-" + event_id}).text.strip()

    event_info["location"] = event.find("span", {"id": "event-location-" + event_id}).text.strip()

    return event_info

def get_length_and_age(event_id) -> tuple[float, float]:
    url = "https://planetarium-hamburg.eventim-inhouse.de/webshop/webticket/seatmap?eventId=23422&el=true"

    headers, cookies = get_headers_and_cookies()

    response = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(response.text, "html.parser")

    #find div.evt-rm-content text-small
    data_div = soup.find("div", {"class": "evt-rm-content text-small"})
    p_tags = data_div.find_all("p")

    length = p_tags[0].text.split("ca.")[-1].split("Min.")[0].strip()

    recomended_age = p_tags[1].text.split("ab ")[-1].split(" Jahren")[0].strip()

    return length, recomended_age

In [4]:
url = "https://planetarium-hamburg.eventim-inhouse.de/webshop/webticket/eventlist"

headers, cookies = get_headers_and_cookies()

response = requests.get(url, headers=headers, cookies=cookies)
soup = BeautifulSoup(response.content, "html.parser")

print(soup.prettify())

print("Status code: ", response.status_code)

#find <ul> id="eventListUl" and get all <li> elements
event_list = soup.find("ul", {"id": "eventListUl"}).find_all("li")


events = {}
for event in event_list:
    # find <h2 id="event-title-EVENTID"> and get text
    title = event.find("h2").text
    event_id = event.find("h2")["id"].split("-")[-1]

    # find span with id "event-date-EVENTID" and get text
    date_span = event.find("span", {"id": "event-date-" + event_id})

    # data_span = <span><span>Fri.</span> 16.02.2024</span>
    weekday = date_span.find("span").text.strip()
    date = date_span.text.split("</span>")[0].split(" ")[-1].strip()

    # find span#event-time-EVENTID.evt-event-detail_time
    time_HH_MM = event.find("span", {"id": "event-time-" + event_id}).text.strip()

    # find span id "event-location-EVENTID" 
    location = event.find("span", {"id": "event-location-" + event_id}).text.strip()

    events[event_id] = {
        "title": title,
        "date": date,
        "weekday": weekday,
        "time": time_HH_MM,
        "location": location
    }


print(events)

<!DOCTYPE html>
<html lang="de">
 <head>
  <title>
   Webshop
  </title>
  <link href="http://www.planetarium-hamburg.de/de/rechtliches/datenschutz" rel="P3Pv1"/>
  <meta charset="utf-8"/>
  <meta content="" name="description"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <script src="/webshop/assets/scripts/vendor/modernizr.js">
  </script>
  <script src="/webshop/assets/scripts/vendor/mobile-detect.js">
  </script>
  <link href="/webshop/images/responsive/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/webshop/assets/styles/bootstrap.css?v=3" rel="stylesheet"/>
  <link href="/webshop/assets/styles/inhouse.css?v=143" rel="stylesheet"/>
  <link href="/webshop/assets/styles/print.css?v=102" media="print" rel="stylesheet"/>
  <link href="/webshop/assets/styles/bootstrap-toggle.min.css" rel="stylesheet"/>
  <link href="/webshop/styles/responsive/design.css?v=113" rel="stylesheet">


AttributeError: 'NoneType' object has no attribute 'find_all'

In [74]:
events = get_event_list()
print(events)

AttributeError: 'NoneType' object has no attribute 'find_all'